In [5]:
import asyncio

- 阻塞
    - 阻塞还是非阻塞：cpu 机器码执行流是否被暂停
    - 阻塞
        - read()：文件io，sleep()，recv()：网络数据接收；
- 核心概念
    - event loop：事件循环
    - 协程（coroutine）对象；

### basics

- asyncio：顾名思义，异步io，处理异步的io（input/output）操作
    - 涉及文件、网络、数据库等需要等待外部资源的情况下
    - asyncio 的底层并不是通过多线程或多进程实现并发，而是通过 单线程的协程 来实现。它使用 事件循环（event loop） 来管理并发任务。协程是一种轻量级的并发方式，能够在单个线程中调度多个任务的执行。
- 协程（coroutine）与 `async`
    - asyncio 使用 Python 中的协程（coroutines），这些协程是通过 async def 定义的函数。当一个协程遇到一个 I/O 操作时（例如网络请求、文件操作等），它会让出控制权，允许事件循环去执行其他任务，而不阻塞主线程。这种机制类似于一种高效的任务切换，而不是创建新的线程或进程。
    - 定义协程函数：使用 `async def` 来定义一个协程函数。
        - 用于定义协程函数，协程是可以在执行过程中暂停的函数。
    - 协程函数在调用时不会立即执行，返回一个协程对象，需要在事件循环（event loop）中运行。
        - 事件循环会调度协程，并在协程让出控制权时执行其他的任务。事件循环使得我们可以在单线程中处理多个任务，避免了线程之间的上下文切换和进程间通信的开销。
        - 它会告诉事件循环 "我需要等待"，事件循环便可以切换去运行其他不需要等待的协程。这种模型在高 I/O 密集场景（如处理大量网络请求）时特别高效。
    - `asyncio.run()` 来运行主协程。
        - 运行最高层级的入口点协程，并管理事件循环的生命周期。
```
# Python 3.7 之前的用法
loop = asyncio.get_event_loop()
loop.run_until_complete(main())
loop.close()
```
- await 关键字
    - 用于等待一个可等待对象（如协程、Future、Task）的结果。
        - 用于等待一个可等待对象的结果，协程在此暂停，直到结果返回。
    - 当协程遇到 await，它会暂停执行，等待结果返回后继续。

In [12]:
async def fetch_data():
    # 模拟耗时操作
    await asyncio.sleep(1)
    return "数据已获取"

In [13]:
async def main():
    result = await fetch_data()
    print(result)

In [14]:
# 返回协程对象
main()

<coroutine object main at 0x788860f2e030>

In [9]:
await main()

数据已获取


In [11]:
import asyncio

async def say_hello():
    await asyncio.sleep(1)
    print("Hello, World!")

async def main():
    await say_hello()

# 运行事件循环
# asyncio.run(main())
await main()

Hello, World!


### 并发执行多个异步任务

In [15]:
async def task(name, delay):
    await asyncio.sleep(delay)
    print(f"Task {name} completed")

async def main():
    tasks = [
        asyncio.create_task(task("A", 2)),
        asyncio.create_task(task("B", 1)),
        asyncio.create_task(task("C", 3))
    ]
    await asyncio.gather(*tasks)

await main()

Task B completed
Task A completed
Task C completed


### `async` 与 `await`

```
# 定义一个异步函数
async def simulate_io_task(delay):
    print(f"任务开始，等待 {delay} 秒...")
    # asyncio.sleep 是异步版本的 time.sleep，但不会阻塞事件循环
    await asyncio.sleep(delay)
    print("任务完成")
    return f"结果：等待了 {delay} 秒"

# 定义另一个异步函数来调用 simulate_io_task
async def main():
    # 等待 simulate_io_task 完成
    result = await simulate_io_task(2)
    print(result)

    # 同时运行多个异步任务，这三个任务总共花了3s；
    task1 = simulate_io_task(1)
    task2 = simulate_io_task(3)
    task3 = simulate_io_task(2)
    results = await asyncio.gather(task1, task2, task3)
    for result in results:
        print(result)

# 运行主函数
if __name__ == "__main__":
    # 使用 asyncio.run 来运行 main 协程
    asyncio.run(main())
```

```
任务开始，等待 2 秒...
任务完成
结果：等待了 2 秒


任务开始，等待 1 秒...
任务开始，等待 3 秒...
任务开始，等待 2 秒...
任务完成
任务完成
任务完成
结果：等待了 1 秒
结果：等待了 3 秒
结果：等待了 2 秒

```